In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pd.set_option('display.max_columns', None)

/kaggle/input/redfin-housing-market-data/zip_code_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/state_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/neighborhood_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/us_national_market_tracker.tsv000
/kaggle/input/redfin-housing-market-data/county_market_tracker.tsv000


In [2]:
# county data df
county_data = '/kaggle/input/redfin-housing-market-data/county_market_tracker.tsv000'
county_df = pd.read_csv(county_data, sep='\t')
county_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,AL,All Residential,-1,120500.0,-0.076628,-0.107077,147900.0,0.006465,0.096368,75.845434,0.053629,0.029213,79.186957,0.010745,0.015406,106.0,-0.109244,-0.131148,48.0,0.000000,-0.058824,140.0,0.196581,0.076923,439.0,0.068127,-0.142578,4.1,0.6,-0.1,101.0,-28.0,-10.0,0.960914,0.009346,0.005058,0.094340,-0.031711,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,MN,Multi-Family (2-4 Unit),4,203000.0,-0.171429,0.052632,210000.0,-0.045020,0.111111,86.633333,-0.171374,0.015114,128.113839,0.025766,0.125353,57.0,0.628571,-0.109375,29.0,0.000000,-0.147059,21.0,-0.562500,-0.447368,133.0,-0.231214,-0.043165,2.3,-2.6,0.1,76.0,31.0,17.0,0.977672,0.000083,-0.004717,0.245614,0.017043,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,OK,All Residential,-1,167500.0,0.021341,0.101974,179700.0,0.057681,0.001114,89.765580,0.017634,0.046460,97.011012,0.021040,0.028369,665.0,0.136752,0.084829,523.0,0.050201,-0.095156,941.0,0.089120,0.012917,2665.0,0.012538,0.002256,4.0,-0.5,-0.3,69.0,15.0,15.0,0.973126,0.004623,0.001848,0.177444,0.001375,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,MN,Condo/Co-op,3,125000.0,-0.137931,NaN,209900.0,NaN,NaN,147.076613,0.014321,NaN,109.934018,NaN,NaN,3.0,2.000000,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,0.250000,NaN,1.7,-2.3,NaN,107.0,-5.0,NaN,0.964538,-0.002774,NaN,0.000000,0.000000,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,MS,Single Family Residential,6,165000.0,-0.144635,-0.046243,181444.0,-0.090278,0.067946,85.000000,-0.159572,-0.102129,95.382115,-0.050421,-0.069443,19.0,0.000000,-0.050000,6.0,-0.333333,-0.400000,18.0,-0.181818,-0.142857,80.0,-0.036145,-0.024390,4.2,-0.2,0.1,117.0,17.0,-36.0,0.981283,0.006887,0.018738,0.157895,-0.052632,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-01,2013-03-31,30,county,5,2728,f,"Gonzales County, TX",NaN,Texas,TX,All Residential,-1,291500.0,2.285210,6.287500,65000.0,-0.641205,-0.671717,83.404488,0.170335,1.145975,44.338336,-0.596514,-0.395622,2.0,0.000000,-0.333333,NaN,NaN,NaN,3.0,-0.400000,-0.250000,28.0,0.000000,0.333333,14.0,0.0,7.0,458.0,431.0,416.0,0.837832,-0.083885,-0.114685,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Texas nonmetropolitan area,NaN,2022-01-09 

In [3]:
# keep only single family homes
sfh_df = county_df[county_df['property_type']=='Single Family Residential']
sfh_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,MS,Single Family Residential,6,165000.0,-0.144635,-0.046243,181444.0,-0.090278,0.067946,85.000000,-0.159572,-0.102129,95.382115,-0.050421,-0.069443,19.0,0.000000,-0.050000,6.0,-0.333333,-0.400000,18.0,-0.181818,-0.142857,80.0,-0.036145,-0.024390,4.2,-0.2,0.1,117.0,17.0,-36.0,0.981283,0.006887,0.018738,0.157895,-0.052632,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
5,2017-06-01,2017-06-30,30,county,5,3227,f,"Walworth County, WI",NaN,Wisconsin,WI,Single Family Residential,6,215500.0,0.051220,0.152406,214000.0,-0.022831,0.021480,125.000000,0.038070,0.111421,129.368932,-0.003066,0.077931,180.0,0.022727,0.065089,109.0,-0.052174,0.252874,203.0,-0.150628,-0.019324,723.0,0.018310,-0.147406,4.0,0.0,-1.0,71.0,-7.0,-18.0,0.977457,0.015774,0.008283,0.188889,0.024116,0.035043,0.199170,0.024522,-0.029603,0.119266,0.032310,0.015818,"Whitewater, WI",48580.0,2022-01-09 14:29:56
7,2018-06-01,2018-06-30,30,county,5,2736,f,"Hamilton County, TX",NaN,Texas,TX,Single Family Residential,6,153000.0,0.285714,0.602094,155000.0,-0.049080,0.275720,51.789077,-0.217643,-0.211508,86.347926,-0.024164,0.078816,5.0,-0.500000,-0.375000,3.0,-0.625000,-0.500000,13.0,0.000000,0.857143,43.0,0.303030,0.048780,8.6,5.3,3.5,20.0,-131.0,-64.0,0.949937,0.014152,0.062323,0.000000,-0.100000,-0.125000,0.232558,0.050740,0.061826,0.666667,0.416667,0.500000,Texas nonmetropolitan area,NaN,2022-01-09 14:29:56
8,2018-07-01,2018-07-31,30,county,5,692,f,"Idaho County, ID",NaN,Idaho,ID,Single Family Residential,6,175000.0,-0.222222,0.080247,205000.0,-0.057471,0.263482,103.550296,0.066039,0.300131,137.012012,0.069474,0.574636,11.0,-0.521739,0.222222,2.0,-0.500000,1.000000,10.0,-0.375000,-0.333333,96.0,-0.020408,0.156627,8.7,4.4,-0.5,222.0,-138.0,-190.0,0.960663,0.023690,-0.019367,0.090909,-0.039526,-0.242424,NaN,NaN,NaN,0.000000,0.000000,0.000000,Idaho nonmetropolitan area,NaN,2022-01-09 14:29:56
10,2017-09-01,2017-09-30,30,county,5,2431,f,"Providence County, RI",NaN,Rhode Island,RI,Single Family Residential,6,234900.0,0.021304,0.145854,229900.0,0.000000,0.100000,142.973856,0.006839,0.093175,137.562680,0.005243,0.117663,445.0,-0.210993,-0.093686,354.0,-0.189931,-0.075718,542.0,-0.115824,0.028463,1450.0,0.013986,-0.123867,3.3,0.8,-0.1,49.0,3.0,-6.0,0.989567,0.000079,0.007426,0.357303,0.041701,0.072171,0.253793,0.009737,0.022373,0.211864,-0.051293,-0.046621,"Providence, RI",39300.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563103,2014-08-01,2014-08-31,30,county,5,1420,f,"Saginaw County, MI",NaN,Michigan,MI,Single Family Residential,6,73625.0,-0.181944,0.036972,79900.0,-0.070390,0.011392,47.095502,-0.196704,-0.023458,56.426887,-0.047781,0.049540,132.0,-0.241379,-0.214286,102.0,-0.113043,-0.120690,225.0,-0.224

In [4]:
# keep only the columns we want 
sfh_columns = sfh_df[['period_begin', 'period_end', 'table_id',  'region',  'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy']]

In [5]:
# data types 
sfh_columns.dtypes

period_begin              object
period_end                object
table_id                   int64
region                    object
state_code                object
property_type             object
property_type_id           int64
median_sale_price        float64
median_sale_price_mom    float64
median_sale_price_yoy    float64
median_list_price        float64
median_list_price_mom    float64
median_list_price_yoy    float64
median_ppsf              float64
median_ppsf_mom          float64
median_ppsf_yoy          float64
median_list_ppsf         float64
median_list_ppsf_mom     float64
median_list_ppsf_yoy     float64
homes_sold               float64
homes_sold_mom           float64
homes_sold_yoy           float64
pending_sales            float64
pending_sales_mom        float64
pending_sales_yoy        float64
new_listings             float64
new_listings_mom         float64
new_listings_yoy         float64
inventory                float64
inventory_mom            float64
inventory_

In [6]:
# see df
sfh_columns

,period_begin,period_end,table_id,region,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy
4,2019-05-01,2019-05-31,1586,"Tate County, MS",MS,Single Family Residential,6,165000.0,-0.144635,-0.046243,181444.0,-0.090278,0.067946,85.000000,-0.159572,-0.102129,95.382115,-0.050421,-0.069443,19.0,0.000000,-0.050000,6.0,-0.333333,-0.400000,18.0,-0.181818,-0.142857,80.0,-0.036145,-0.024390,4.2,-0.2,0.1,117.0,17.0,-36.0,0.981283,0.006887,0.018738,0.157895,-0.052632,0.107895
5,2017-06-01,2017-06-30,3227,"Walworth County, WI",WI,Single Family Residential,6,215500.0,0.051220,0.152406,214000.0,-0.022831,0.021480,125.000000,0.038070,0.111421,129.368932,-0.003066,0.077931,180.0,0.022727,0.065089,109.0,-0.052174,0.252874,203.0,-0.150628,-0.019324,723.0,0.018310,-0.147406,4.0,0.0,-1.0,71.0,-7.0,-18.0,0.977457,0.015774,0.008283,0.188889,0.024116,0.035043
7,2018-06-01,2018-06-30,2736,"Hamilton County, TX",TX,Single Family Residential,6,153000.0,0.285714,0.602094,155000.0,-0.049080,0.275720,51.789077,-0.217643,-0.211508,86.347926,-0.024164,0.078816,5.0,-0.500000,-0.375000,3.0,-0.625000,-0.500000,13.0,0.000000,0.857143,43.0,0.303030,0.048780,8.6,5.3,3.5,20.0,-131.0,-64.0,0.949937,0.014152,0.062323,0.000000,-0.100000,-0.125000
8,2018-07-01,2018-07-31,692,"Idaho County, ID",ID,Single Family Residential,6,175000.0,-0.222222,0.080247,205000.0,-0.057471,0.263482,103.550296,0.066039,0.300131,137.012012,0.069474,0.574636,11.0,-0.521739,0.222222,2.0,-0.500000,1.000000,10.0,-0.375000,-0.333333,96.0,-0.020408,0.156627,8.7,4.4,-0.5,222.0,-138.0,-190.0,0.960663,0.023690,-0.019367,0.090909,-0.039526,-0.242424
10,2017-09-01,2017-09-30,2431,"Providence County, RI",RI,Single Family Residential,6,234900.0,0.021304,0.145854,229900.0,0.000000,0.100000,142.973856,0.006839,0.093175,137.562680,0.005243,0.117663,445.0,-0.210993,-0.093686,354.0,-0.189931,-0.075718,542.0,-0.115824,0.028463,1450.0,0.013986,-0.123867,3.3,0.8,-0.1,49.0,3.0,-6.0,0.989567,0.000079,0.007426,0.357303,0.041701,0.072171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563103,2014-08-01,2014-08-31,1420,"Saginaw County, MI",MI,Single Family Residential,6,73625.0,-0.181944,0.036972,79900.0,-0.070390,0.011392,47.095502,-0.196704,-0.023458,56.426887,-0.047781,0.049540,132.0,-0.241379,-0.214286,102.0,-0.113043,-0.120690,225.0,-0.224138,-0.150943,1024.0,0.014866,0.054583,7.8,2.0,2.0,62.0,13.0,19.0,0.908060,-0.029699,-0.041529,0.106061,-0.003135,-0.030844
563105,2012-10-01,2012-10-31,1982,"Oswego County, NY",NY,Single Family Residential,6,96500.0,0.072222,0.034853,79900.0,-0.200200,-0.076301,64.296721,0.058784,0.055553,65.454545,0.065205,0.167884,88.0,0.173333,0.375000,28.0,0.473684,1.153846,77.0,-0.163043,-0.125000,622.0,-0.091971,-0.181579,7.1,-2.0,-4.8,132.0,-9.0,-3.0,0.944614,0.000786,0.014577,0.193182,-0.020152,0.052557
563110,2020-06-01,2020-06-30,3219,"Sauk County, WI",WI,Single Family Residential,6,244900.0,0.014289,0.272208,220900.0,-0.067736,-0.035371,125.218659,0.139813,0.195517,126.937782,-0.023180,0.093419,75.0,0.102941,-0.226804,60.0,0.200000,-0.200000,125.0,0.582278,0.288660,264.0,0.168142,-0.161905,3.5,0.2,0.3,55.0,-7.0,-11.0,0.974688,0.002437,-0.006419,0.213333,0.125098,-0.054708
563115,2019-12-01,2019-12-31,2900,"Duchesne County, UT",UT,Single Family Residential,6,2

In [7]:
# to csv 
sfh_columns.to_csv('sfh_columns.csv')

In [8]:
# state data df 
state_data = '/kaggle/input/redfin-housing-market-data/state_market_tracker.tsv000'
state_df = pd.read_csv(state_data, sep='\t')
state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-10-01,2019-10-31,30,state,4,23,f,Oklahoma,NaN,Oklahoma,OK,Multi-Family (2-4 Unit),4,162200,-0.164611,0.066970,185000.0,-0.000173,0.005355,77.0,-0.243652,0.000027,107.0,0.010743,0.039438,42,-0.322581,-0.023256,47.0,-0.060000,0.175000,39.0,-0.093023,-0.187500,81.0,-0.147368,-0.295652,1.9,0.4,-0.8,45.0,9.0,23.0,0.990685,0.020968,0.028253,0.238095,0.141321,0.098560,0.209877,0.083561,0.053355,0.301282,-0.240768,-0.122115,South Region,NaN,2022-01-09 14:29:56
1,2021-07-01,2021-07-31,30,state,4,40,f,Vermont,NaN,Vermont,VT,All Residential,-1,317900,-0.027286,0.204762,322700.0,-0.011254,0.073241,177.0,0.008954,0.209023,183.0,-0.022605,0.164172,1049,-0.038497,-0.074139,909.0,0.043628,0.139098,1307.0,0.013964,-0.091099,2760.0,0.037594,-0.270034,2.6,0.2,-0.7,49.0,-2.0,-40.0,1.007921,-0.003433,0.033684,0.487131,0.006837,0.251473,0.133696,0.016402,-0.000131,0.424404,-0.064422,0.076156,Northeast Region,NaN,2022-01-09 14:29:56
2,2016-08-01,2016-08-31,30,state,4,10,f,New Hampshire,NaN,New Hampshire,NH,Condo/Co-op,3,200100,0.011560,0.089445,260200.0,-0.029147,0.099136,155.0,0.020958,0.110774,145.0,-0.005569,0.080314,276,0.189655,0.045455,200.0,0.104972,0.010101,304.0,0.055556,-0.081571,1148.0,-0.041736,-0.217984,4.2,-1.0,-1.4,71.0,5.0,-9.0,0.978521,-0.005818,0.005599,0.275362,0.021052,0.089756,0.149826,0.009592,0.002687,0.206101,0.061385,0.016779,Northeast Region,NaN,2022-01-09 14:29:56
3,2013-04-01,2013-04-30,30,state,4,51,f,Mississippi,NaN,Mississippi,MS,All Residential,-1,129500,0.057733,0.071354,144200.0,0.035380,0.053684,69.0,0.002417,0.012518,79.0,0.013194,0.036259,1008,0.161290,0.204301,403.0,0.271293,0.144886,1343.0,-0.031723,0.056648,6457.0,0.002328,0.204215,6.4,-1.0,0.0,115.0,-8.0,17.0,0.946311,0.004240,0.001599,0.112103,-0.014625,-0.008566,NaN,NaN,NaN,0.044777,-0.014869,-0.005006,South Region,NaN,2022-01-09 14:29:56
4,2019-12-01,2019-12-31,30,state,4,2,f,Missouri,NaN,Missouri,MO,Condo/Co-op,3,152000,-0.016941,0.067996,170100.0,-0.061422,0.056845,135.0,0.028679,0.076636,122.0,-0.043701,0.078837,312,0.036545,0.126354,NaN,NaN,NaN,163.0,-0.391791,-0.094444,487.0,-0.157439,-0.263238,1.6,-0.3,-0.8,47.0,11.0,-14.0,0.974024,-0.004532,0.005082,0.131274,-0.063945,-0.017031,0.205339,-0.123381,-0.000410,0.252082,-0.116423,0.095007,Midwest Region,NaN,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,2012-03-01,2012-03-31,30,state,4,6,f,Nebraska,NaN,Nebraska,NE,All Residential,-1,133000,0.046958,0.035267,148900.0,0.038652,0.037200,74.0,0.028385,0.021285,80.0,-0.007782,0.028330,1087,0.487004,0.355362,1142.0,0.662300,0.657475,2178.0,0.508310,0.263341,5133.0,0.074299,-0.004847,4.7,-1.8,-1.7,76.0,-9.0,-17.0,0.962492,-0.002366,0.011467,0.174793,-0.011254,0.021426,NaN,NaN,NaN,0.422780,0.042801,0.030384,Midwest Region,NaN,2022-01-09 14:29:56
27074,2016-07-01,2016-07-31,30,state,4,27,f,Idaho,Na

In [9]:
# national market data
national_market_data = '/kaggle/input/redfin-housing-market-data/us_national_market_tracker.tsv000'
national_df = pd.read_csv(national_market_data, sep='\t')
national_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2013-09-01,2013-09-30,30,national,11,1400,t,National,National,U.S.,US,All Residential,-1,209749.175328,0.001382,0.122263,219272.689364,-0.001089,0.116345,114.368524,-0.000961,0.113057,118.801761,0.007225,0.108324,443966.614345,-0.025552,0.092768,329884.763819,-0.010884,0.194803,569804.074196,0.001327,0.127461,1.859411e+06,0.007445,-0.006764,4.187197,0.196740,-0.394015,61.196654,0.0,-14.0,0.975328,-0.000380,0.005876,0.220366,-0.003847,0.014642,0.062155,0.003375,0.010655,NaN,NaN,NaN,NaN,NaN,2022-01-09 00:00:00
1,2012-04-01,2012-04-30,30,national,11,1400,f,National,National,U.S.,US,All Residential,-1,177995.000000,0.041082,0.059830,199160.000000,0.000236,0.005418,98.000000,0.039573,0.043827,108.000000,-0.000916,-0.007283,402984.000000,0.026877,0.085324,307367.000000,0.085013,0.221988,604361.000000,-0.024723,-0.018376,2.145026e+06,0.017074,-0.164411,5.322848,-0.051305,-1.590852,81.000000,-12.0,-16.0,0.965322,0.003403,0.006966,0.188318,0.005978,0.018905,0.055948,-0.003248,-0.017449,0.325238,-0.008578,0.062488,NaN,NaN,2022-01-09 14:33:38
2,2018-09-01,2018-09-30,30,national,11,1400,t,National,National,U.S.,US,Single Units Only,-2,NaN,NaN,NaN,295969.767699,0.014295,0.055043,151.477408,0.001997,0.042456,158.576424,0.008698,0.052866,NaN,NaN,NaN,424690.502061,-0.020755,0.000495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.149808,0.0,-3.0,0.983206,-0.000115,-0.000397,NaN,NaN,NaN,0.149476,0.002931,0.030961,NaN,NaN,NaN,NaN,NaN,2022-01-09 00:00:00
3,2019-07-01,2019-07-31,30,national,11,1400,f,National,National,U.S.,US,All Residential,-1,299192.000000,-0.009805,0.043091,299935.000000,-0.024507,0.030800,158.000000,-0.004935,0.029624,160.000000,-0.008994,0.021545,637980.000000,0.020104,0.025806,543419.000000,0.012570,0.007684,706482.000000,-0.043400,-0.014060,1.683965e+06,-0.008848,0.013417,2.639524,-0.077102,-0.032268,36.000000,1.0,4.0,0.985874,-0.001017,-0.001881,0.253263,-0.009679,-0.025856,0.167575,0.006934,0.006512,0.358500,-0.037603,-0.016629,NaN,NaN,2022-01-09 14:33:38
4,2013-01-01,2013-01-31,30,national,11,1400,t,National,National,U.S.,US,Condo/Co-op,3,162698.342202,-0.005120,0.141005,169134.622288,0.019795,0.077148,NaN,NaN,NaN,143.234637,0.011457,0.070786,NaN,NaN,NaN,37757.201062,0.040593,0.201611,NaN,NaN,NaN,2.107168e+05,-0.007550,-0.231386,4.304467,0.151052,-1.659854,64.519161,-5.0,-22.0,NaN,NaN,NaN,0.231539,0.013082,0.059250,0.060066,-0.029603,-0.015023,0.351386,-0.010496,0.105331,NaN,NaN,2022-01-09 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,2021-11-01,2021-11-30,30,national,11,1400,f,National,National,U.S.,US,Condo/Co-op,3,309323.000000,0.023331,0.113943,307909.000000,-0.020030,0.093872,274.000000,0.019776,0.153580,276.000000,-0.019413,0.116354,65408.000000,-0.034210,0.064920,58752.000000,-0.090920,0.089925,53689.000000,-0.227308,-0.075693,1.046140e+05,-0.149938,-0.387505,1.599389,-0.2

In [10]:
# look at each month, by each metro region, for each year - see percent growth of median sale price, etc